In [36]:
import pandas as pd
import os
import re
from collections import defaultdict

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pymorphy3 import MorphAnalyzer

RE_DERIVED = re.compile(r"\w+( -|- |-|! - )\w+")
RE_ENGLISH_TEXT = re.compile(r"[а-яА-Яa-zA-Z0-9\-]+")

nltk.download('stopwords')

stopwords_ru = stopwords.words("english")


# Topics
from typing import Optional
from typing import Union

import copy
import pickle

from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import numpy as np



morph = MorphAnalyzer()

def parse_pages_to_paragraphs(files: list[str], rep_year):
    errors = []
    df = []
    report_idx = 0
    for report_name in tqdm(files):
        if not report_name.endswith(".txt"):
            print(report_name)
            continue
        try:
            #report = textract.process(report_name)
            report = open(report_name, 'r', encoding='utf-8').read()
        except Exception as e:
            errors.append((report_name, e))
            continue

        # Регулярное выражение для извлечения имени компании и номера отчета

        company_name = os.path.basename(report_name)
        report_index = os.path.basename(report_name)

        for i, paragraph in enumerate(report.split("\n")):
            if len(paragraph)>500:
                paragraph = paragraph.replace("\n", " ").lower()
                report_page = RE_DERIVED.sub("", paragraph)
                report_page_lst = word_tokenize(report_page)
                # russian_report_page_lst = [w for w in filter(RE_RUSSIAN_TEXT.match, report_page_lst)]
                tokens = []
                # nouns = set()
                # verbs = set()
                # adj = set()
                for word_ in filter(RE_ENGLISH_TEXT.match, report_page_lst):
                    if word_ and word_ not in stopwords_ru and word_.isalpha():
                        word_ = word_.strip()
                        word_ = morph.parse(word_)[0]
                        normal_form = word_.normal_form
                        tokens.append(normal_form)
                df.append(
                    {
                    "company": company_name,
                    "paragraph": i,
                    "original_text": paragraph,
                    "cleaned_text": tokens,
                    "year": rep_year
                    }
                )

    print(*errors, sep="\n")
    return df

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\avang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
import pandas as pd
import numpy as np

df = pd.read_csv('C:\\Users\\avang\\OneDrive\\Документы\\Thesis\\Thesis lab\\classified_withi_ccc.csv')
df.rename(columns={'Unnamed: 0': 'Company'}, inplace=True)
df.columns

Index(['Company', 'Business_Ethics', 'Data_Security',
       'Access_And_Affordability', 'Business_Model_Resilience',
       'Competitive_Behavior', 'Critical_Incident_Risk_Management',
       'Customer_Welfare', 'Director_Removal',
       'Employee_Engagement_Inclusion_And_Diversity',
       'Employee_Health_And_Safety', 'Human_Rights_And_Community_Relations',
       'Labor_Practices', 'Management_Of_Legal_And_Regulatory_Framework',
       'Physical_Impacts_Of_Climate_Change', 'Product_Quality_And_Safety',
       'Product_Design_And_Lifecycle_Management',
       'Selling_Practices_And_Product_Labeling', 'Supply_Chain_Management',
       'Systemic_Risk_Management', 'Waste_And_Hazardous_Materials_Management',
       'Water_And_Wastewater_Management', 'Air_Quality', 'Customer_Privacy',
       'Ecological_Impacts', 'Energy_Management', 'GHG_Emissions'],
      dtype='object')

In [ ]:
df.drop(df.loc[df['Company'] == 'Porsche'].index, inplace=True)

In [43]:
df.drop(df.loc[df['Company'] == 'Philips'].index, inplace=True)
df.reset_index(inplace=True)

In [45]:
df.drop(columns=['index'], inplace=True)
df

,Company,Business_Ethics,Data_Security,Access_And_Affordability,Business_Model_Resilience,Competitive_Behavior,Critical_Incident_Risk_Management,Customer_Welfare,Director_Removal,Employee_Engagement_Inclusion_And_Diversity,...,Selling_Practices_And_Product_Labeling,Supply_Chain_Management,Systemic_Risk_Management,Waste_And_Hazardous_Materials_Management,Water_And_Wastewater_Management,Air_Quality,Customer_Privacy,Ecological_Impacts,Energy_Management,GHG_Emissions
0,Amlogic,['Report Reliability Guarantee The company gua...,['Amlogic places a high priority on informatio...,[],[],['center. This will enable us to continually i...,"['In December 2022, the Company organized a fi...",[],"['5 members including Independent Directors', ...","['2023 Environmental, Social, and Governance E...",...,[],"['2023 Environmental, Social, and Governance E...","['2023 Environmental, Social, and Governance E...","['Prior to holidays, announcements are made to...","['6,721cubic meters of water, a decrease of ap...",[],['Materials containing the personal informatio...,"['2023 Environmental, Social, and Governance E...",['decoder and ultralow power consumption Amlog...,[]
1,AviChina,['Consummation of Corporate Governance Active ...,['Linkage to departmental performance IV. Indu...,"['During the reporting period, subsidiaries ac...",['The Board of Directors of the Group attaches...,"['n an icon is on a square, that square must b...",['Support Rural Revitalization II Emergency Re...,[],['Statement of The Board The Board of Director...,['Diversity of Directors Diversity of board me...,...,[],['Driven by Technological Innovation II Highqu...,"['In terms of risk management, the Company con...",['1. Wastewater and Exhaust Gas Management Acc...,['Chronic risk Continued average temperature i...,['Each subsidiary formulates practicable carbo...,['All data and information disclosed in this r...,"['BEAUTIFUL ENVIRONMENT FOR FREE', 'Social In ...","['By 2030, we will benchmark against the green...",['Financial performance Indicator Total assets...
2,Dino,"['Accordingly, it is exposed to the risk of lo...","['The Dino Polska Groups IT systems may fail, ...","['Moreover, Dino Polska reports that it does n...",['7.1. DINO POLSKAS BUSINESS MODEL AND GROWTH ...,['the consent of the President of the Office o...,['In 2022 the accident ratio the number of eve...,[],['6.6. Management Board of Dino Polska Members...,['Standard 2.1 The Company should have a dive...,...,['In 2023 we donated a total of 625 tons of fo...,['Suppliers The Dino Polska Group collaborates...,['6.2. INTERNAL CONTROL AND RISK MANAGEMENT SY...,['The execution of Dino Polskas environmental ...,['Water consumption and production of wastewat...,"['tons of CO2e in 2022, which is made up of th...",['The Company has a Personal Data Protection P...,['In the pursuit of its business objectives Di...,"['83 MW, up by 31 from the end of 2022. This e...","['Direct greenhouse gas emissions scope 1', 'I..."
3,Getinge,['Percentage of employees who completed traini...,['Risks at Getinge are managed in accordance w...,"['At Getinge, we work globally with a shared b...",['1315 Getinges strategy and vision beyond 203...,"['Accordingly, innovation beyond 2030 includes...","['Servo ventilators get a virtual twin, TwinVi...",[],['www.getinge.com. The Articles of Association...,['For our business to evolve and to achieve ou...,...,['All of Getinges products are evaluated as re...,['Environment We work actively to reduce negat...,"['4752 Assurance framework, risks and risk man...",['5 of carbon emissions worldwide and also gen...,['Identification of actual and potential impac...,['Transportation Transporting materials betwee...,"['METHODOLOGIES TO REPORT EMPLOYEE DATA', '135...",['By minimizing environmental impacts and supp...,['This is based on a detailed roadmap and the ...,"['and 2 emissions by', 'emissions by 25', 'emi..."
4,Meta,['The chart below shows the distribution of au...,[],['STEM education More than 1 million in

In [46]:
import ast
from tqdm import tqdm  # for progress bar

def convert_string_to_list(cell):
    """Safely convert string representation of list to actual list"""
    if pd.isna(cell):
        return []  # Handle NaN values
    if isinstance(cell, list):
        return cell  # Already a list
    if isinstance(cell, str):
        try:
            # First try literal eval for Python-style lists
            return ast.literal_eval(cell.strip())
        except (ValueError, SyntaxError):
            try:
                # Fallback for JSON-style lists
                import json
                return json.loads(cell.replace("'", '"'))
            except:
                return []  # Return empty list if all conversions fail
    return []  # Default for other types

# Create a copy of your original DataFrame
df_clean = df.copy()

# Convert all columns except the first (company names)
print("Converting string-encoded lists to actual lists...")
for col in tqdm(df_clean.columns[1:]):  # Skip first column
    df_clean[col] = df_clean[col].apply(convert_string_to_list)

# Verify conversion
print("\nConversion complete. Sample verification:")
#print(df_clean.iloc[0, 1])  # Should show actual list now, not string

# Save to new file (multiple format options)
# df_clean.to_pickle('dataframe_with_lists.pkl')  # Best for preserving data types

Converting string-encoded lists to actual lists...


100%|██████████| 26/26 [00:00<00:00, 1535.21it/s]


Conversion complete. Sample verification:


In [47]:
def concatenate_sentences(cell):
    """Convert list of strings to single string with '. ' separation"""
    if isinstance(cell, list):
        # Join with '. ' and ensure proper sentence formatting
        return '. '.join(str(item).strip() for item in cell if str(item).strip()) + ('.' if cell else '')
    return cell  # Return as-is if not a list

# Apply to all cells except first column (company names)
df_concatenated = df_clean.copy()
for col in df_concatenated.columns[1:]:
    df_concatenated[col] = df_concatenated[col].apply(concatenate_sentences)

# Verify results
print("Before conversion (sample cell):")
print(df_clean.iloc[0, 1])  # Original list

print("\nAfter conversion (same cell):")
print(df_concatenated.iloc[0, 1])  # Concatenated string

Before conversion (sample cell):
['Report Reliability Guarantee The company guarantees the veracity, accuracy, and completeness of the Report, with no false records, misleading statements, or significant omissions.', 'Stakeholder Identification Expectations and Demands Environmental Issues Stockholder Governance Issues Corporate governance Information disclosure Anticorruption and integrity Risk management Intellectual property Information security Environmental management Green operations Social Issues Scientific research innovation Quality control Digital intelligence leadership Employee development Industry collaboration Social contribution Customer Highquality products Aftersales service Privacy protection Scientific research innovation Highquality service Protecting customer privacy security Employees Compensation and benefits Protection of employee rights and interests Employee training and career development Employee health and safety Industry Substantive Issue Analysis Communit

In [48]:
import pickle
# Load (next session)
with open('matrix_prob.pkl', 'rb') as f:
    matrix_prob = pickle.load(f)

In [49]:
classes = ['Business_Ethics', 'Data_Security',
       'Access_And_Affordability', 'Business_Model_Resilience',
       'Competitive_Behavior', 'Critical_Incident_Risk_Management',
       'Customer_Welfare', 'Director_Removal',
       'Employee_Engagement_Inclusion_And_Diversity',
       'Employee_Health_And_Safety', 'Human_Rights_And_Community_Relations',
       'Labor_Practices', 'Management_Of_Legal_And_Regulatory_Framework',
       'Physical_Impacts_Of_Climate_Change', 'Product_Quality_And_Safety',
       'Product_Design_And_Lifecycle_Management',
       'Selling_Practices_And_Product_Labeling', 'Supply_Chain_Management',
       'Systemic_Risk_Management', 'Waste_And_Hazardous_Materials_Management',
       'Water_And_Wastewater_Management', 'Air_Quality', 'Customer_Privacy',
       'Ecological_Impacts', 'Energy_Management', 'GHG_Emissions']

In [50]:
import math

# PMI calculation function
def compute_pmi(word1, word2, word_probabilities, pair_probabilities, epsilon=1e-9):
    p_word1 = word_probabilities.get(word1, 0)
    p_word2 = word_probabilities.get(word2, 0)
    p_pair = pair_probabilities.get(tuple(sorted((word1, word2))), 0)

    if p_word1 == 0 or p_word2 == 0:
        return 0  # Если одно из слов не встречается, PMI = 0
    pmi = math.log((p_pair + epsilon) / (p_word1 * p_word2))


    return pmi

# Coherence calculation function
def compute_coherence(text, word_probabilities, pair_probabilities):
    N = len(text)
    if N < 2:
        return 0  # Когерентность не определена для текстов с менее чем 2 словами

    total_pmi = 0
    for i in range(N - 1):
        for j in range(i + 1, N):
            total_pmi += compute_pmi(text[i], text[j], word_probabilities, pair_probabilities)

    coherence = (2 / (N * (N - 1))) * total_pmi
    return coherence

In [85]:
%%time
df_ref = []
stuff = []

''' 'Business_Ethics', 'Data_Security',
       'Access_And_Affordability', 'Business_Model_Resilience',
       'Competitive_Behavior', 'Critical_Incident_Risk_Management',
       'Customer_Welfare', 'Director_Removal',
       'Employee_Engagement_Inclusion_And_Diversity',
       'Employee_Health_And_Safety', 'Human_Rights_And_Community_Relations',
       'Labor_Practices', 'Management_Of_Legal_And_Regulatory_Framework',
       'Physical_Impacts_Of_Climate_Change', 'Product_Quality_And_Safety',
       'Product_Design_And_Lifecycle_Management',
       'Selling_Practices_And_Product_Labeling', 'Supply_Chain_Management',
       'Systemic_Risk_Management', 'Waste_And_Hazardous_Materials_Management',
       'Water_And_Wastewater_Management', 'Air_Quality', 'Customer_Privacy',
       'Ecological_Impacts', 'Energy_Management', 'GHG_Emissions' '''


'''for sasbclass in tqdm(classes):
    for item in list(df_clean[sasbclass]):
        for para in item:
            for el in matrix_prob:
                df_ref.append([el, compute_coherence(para, matrix_prob[el][0],matrix_prob[el][1])])'''
cnt = 0
companies = []
for i in range(len(df_concatenated.GHG_Emissions)):
    file = df_concatenated.GHG_Emissions[i]
    file = file.split(' ')
    print(cnt)
    for el in matrix_prob:
        df_ref.append([el, compute_coherence(file, matrix_prob[el][0], matrix_prob[el][1])])
        stuff.append(compute_coherence(file, matrix_prob[el][0], matrix_prob[el][1]))
        companies.append(df_concatenated['Company'][i])
    cnt += 1

print(stuff)

df_ref = pd.DataFrame(df_ref, columns = ['Class', 'Coherence_value'])
df_ref['company'] = companies
df_ref.head()

0
1
2
3
4
5
6
7
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.14044099937816665, -0.1878745091521992, -0.24016980293650497, -0.22631109673758942, -0.3698585054365209, -0.34147315229416664, -0.08637975392456818, -0.0844838041121353, -0.09283859558055584, -0.14368790535259285, -0.14063645626754673, -0.19856444720308647, -0.08272696448247353, -0.12910660396599816, -0.27263323964523467, -0.07868332707776715, -0.34025678875693577, -0.15315460159182764, -0.19636018275801775, -0.22991715188114303, -0.22294013157885811, -0.33359413184226844, -0.1449067449455817, -0.2510122531200939, -0.02603735417108777, -0.03318250297855553, -0.18385163737717278, -0.2645844999538452, -0.307841250289919, -0.3024356240738777, -0.4232943140173402, -0.42528717396800386, -0.045200876892550396, -0.14678633751514097, -0.09452710913306618, -0.22417683239362632, -0.17640042666263525, -0.1960973080761945, -0.11482470226544658, -0.24828822081591978, -0.28578332958874303, -0.2225279549

,Class,Coherence_value,company
0,Business_Ethics,0.0,Amlogic
1,Data_Security,0.0,Amlogic
2,Access_And_Affordability,0.0,Amlogic
3,Business_Model_Resilience,0.0,Amlogic
4,Competitive_Behavior,0.0,Amlogic


In [86]:
((df_ref.loc[df_ref['Class'] == 'GHG_Emissions']).reset_index()).to_csv('C:\\Users\\avang\\OneDrive\\Документы\\Thesis\\Thesis lab\\CCC_res\\GHG_Emissions.csv')

In [ ]:
df_ref

,Class,Coherence_value,company
0,Business_Ethics,0.0,Amlogic
1,Data_Security,0.0,Amlogic
2,Access_And_Affordability,0.0,Amlogic
3,Business_Model_Resilience,0.0,Amlogic
4,Competitive_Behavior,0.0,Amlogic
...,...,...,...
255,Air_Quality,0.0,Supermicro
256,Customer_Privacy,0.0,Supermicro
257,Ecological_Impacts,0.0,Supermicro
258,Energy_Management,0.0,Supermicro


In [43]:
dfdf = pd.read_csv('BBB_res/Business_Ethics.csv')
dfdf

,Unnamed: 0,index,Class,Coherence_value,company
0,0,0,Business_Ethics,-0.132830,AAC
1,1,26,Business_Ethics,-0.149155,Corning
2,2,52,Business_Ethics,-0.092401,NaN
3,3,78,Business_Ethics,-0.150441,NaN
4,4,104,Business_Ethics,-0.146022,NaN
5,5,130,Business_Ethics,-0.147997,NaN
6,6,156,Business_Ethics,-0.154618,NaN
7,7,182,Business_Ethics,-0.155580,NaN
8,8,208,Business_Ethics,-0.147910,NaN
9,9,234,Business_Ethics,-0.135065,NaN


In [40]:
print(df['Company'])

0           AAC
1         ADNOC
2           AGC
3       Albilad
4         Aldar
5       Alibaba
6        Alinma
7      Alphabet
8        Amazon
9      Amphenol
10        Anker
11      AOSmith
12        Apple
13         Ares
14         Asia
15       ATandT
16     AutoZone
17      Azrieli
18       Balder
19        Banco
20      Barrick
21       Baxter
22          BCE
23      BeiGene
24     Brenntag
25      Comcast
26      Corning
27      eMemory
28          Fox
29     Glencore
30       Impala
31      Jiangsu
32     Marathon
33          MMG
34          MPS
35          MSI
36      Northam
37     Reliance
38       Remgro
39      Rollins
40    Sinopharm
41      Walmart
42       Watsco
Name: Company, dtype: object
